In [1]:
pip install pandas openpyxl sqlalchemy psycopg2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
# Read the excel file pre-Processed

df = pd.read_excel("traffic_stops_cleaned data.xlsx")
df.head()

,country_name,driver_gender,driver_age,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,stop_datetime
0,Canada,Male,19,Speeding,True,Vehicle Search,Ticket,True,16-30 min,True,UP76DY3473,2020-01-01 00:00:00
1,India,Male,58,Other,False,Vehicle Search,Arrest,True,16-30 min,True,RJ83PZ4441,2020-01-01 00:01:00
2,USA,Male,76,Speeding,False,Frisk,Ticket,True,16-30 min,True,RJ32OM7264,2020-01-01 00:02:00
3,Canada,Male,76,DUI,True,Frisk,Warning,False,0-15 min,True,RJ76TI3807,2020-01-01 00:03:00
4,Canada,Male,75,Other,False,Vehicle Search,Arrest,True,16-30 min,False,WB63BB8305,2020-01-01 00:04:00


In [27]:
# Connect to PostgreSQL with cleaned data

from sqlalchemy import String, Integer, Boolean, DateTime

# PostgreSQL connection details
username = "postgres"
password = "malai123"
host = "localhost"
port = "5432"
database = "postgres"

"""Create connection string (+psycopg2: Specifies that the driver used to connect & postgresql: This tells 
SQLAlchemy you are connecting to a PostgreSQL database.)"""

connection_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Upload data to PostgreSQL
table_name = "traffic_logs"  # You can change this
df.to_sql(table_name, engine, if_exists='replace', index=False)

538

In [28]:
df.columns

Index(['country_name', 'driver_gender', 'driver_age', 'violation',
       'search_conducted', 'search_type', 'stop_outcome', 'is_arrested',
       'stop_duration', 'drugs_related_stop', 'vehicle_number',
       'stop_datetime'],
      dtype='object')

In [30]:
# After check in the database all columns are ok but the data types are miss matched has to change
# replace in PostgreSQL
df.to_sql(
    'traffic_logs',
    engine,
    if_exists='replace',
    index=False,
    dtype={
        'country_name': String(15),
        'driver_gender': String(10),
        'driver_age': Integer(),
        'violation': String(20),
        'search_conducted': Boolean(),
        'search_type': String(25),
        'stop_outcome': String(20),
        'is_arrested': Boolean(),
        'stop_duration': String(25),
        'drugs_related_stop': Boolean(),
        'vehicle_number': String(20),
        'stop_datetime': DateTime()
    }
)

538

### MEDIUM LEVEL SQL QUERIES:
 

In [ ]:
# 1.What are the top 10 vehicle_Number involved in drug-related stops?
query = """
SELECT vehicle_number, COUNT(*) AS stop_count
FROM traffic_logs
WHERE drugs_related_stop = TRUE
GROUP BY vehicle_number
ORDER BY stop_count DESC
LIMIT 10;
"""

# Execute
results = pd.read_sql(query, engine)

print(results)

  vehicle_number  stop_count
0     TN16EN4769           1
1     GJ99TA8565           1
2     MH85NR6252           1
3     RJ88QF9639           1
4     WB82DO3310           1
5     TN98SO9644           1
6     RJ69QZ5424           1
7     RJ79BO9666           1
8     TN57CN7350           1
9     TN72BR3512           1


In [ ]:
# 2. Which vehicles were most frequently searched?
query = """
SELECT vehicle_number, COUNT(*) AS search_count
FROM traffic_logs
WHERE search_conducted = TRUE
GROUP BY vehicle_number
ORDER BY search_count DESC
LIMIT 10;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  vehicle_number  search_count
0     TN24KB4148             1
1     TN16EN4769             1
2     MH85NR6252             1
3     KA36NX6664             1
4     WB82DO3310             1
5     RJ14TR4444             1
6     RJ49OQ1668             1
7     MH21PZ2993             1
8     RJ69QZ5424             1
9     TN72BR3512             1


In [ ]:
# 3. Which driver age group had the highest arrest rate?
query = """
SELECT
    CASE
        WHEN driver_age < 25 THEN 'Under 25'
        WHEN driver_age BETWEEN 25 AND 40 THEN '25-40'
        ELSE 'Over 40'
    END AS age_group,
    ROUND(AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100, 2) AS arrest_rate_percentage
FROM traffic_logs
GROUP BY age_group
ORDER BY arrest_rate_percentage DESC
LIMIT 1;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)


  age_group  arrest_rate_percentage
0  Under 25                   51.15


In [34]:
# 4. What is the gender distribution of drivers stopped in each country?
query = """
SELECT
    country_name,
    driver_gender,
    COUNT(*) AS total_count
FROM traffic_logs
GROUP BY country_name, driver_gender
ORDER BY country_name, total_count DESC;
"""

#  Run the query
results = pd.read_sql(query, engine)

print(results)


  country_name driver_gender  total_count
0       Canada        Female        10997
1       Canada          Male        10911
2        India        Female        11043
3        India          Male        10955
4          USA        Female        10841
5          USA          Male        10791


In [ ]:
# 5. Which race and gender combination has the highest search rate?
query = """
SELECT
    driver_gender,
    ROUND(AVG(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) * 100, 2) AS search_rate_percentage
FROM traffic_logs
GROUP BY driver_gender
ORDER BY search_rate_percentage DESC
LIMIT 2;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  driver_gender  search_rate_percentage
0        Female                   50.14
1          Male                   49.66


In [40]:
# 6. What time of day sees the most traffic stops?
query = """
SELECT
    EXTRACT(HOUR FROM stop_datetime) AS hour_of_day,
    COUNT(*) AS stop_count
FROM traffic_logs
GROUP BY hour_of_day
ORDER BY stop_count DESC
LIMIT 1;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

   hour_of_day  stop_count
0          8.0        2760


In [ ]:
# 7. What is the average stop duration for different violations?
query = """
SELECT
    violation,
    ROUND(AVG(duration_minutes)::numeric, 2) AS avg_stop_minutes
FROM (
    SELECT
        violation,
        CASE 
            WHEN stop_duration ~ '^[0-9]+-[0-9]+' THEN
                (
                    CAST(split_part(stop_duration, '-', 1) AS FLOAT) + 
                    CAST(split_part(split_part(stop_duration, ' ', 1), '-', 2) AS FLOAT)
                ) / 2
            WHEN stop_duration ~ '^[0-9]+' THEN
                CAST(regexp_replace(stop_duration, '[^0-9]', '', 'g') AS FLOAT)
            ELSE NULL
        END AS duration_minutes
    FROM traffic_logs
    WHERE stop_duration IS NOT NULL
) AS parsed
WHERE duration_minutes IS NOT NULL
GROUP BY violation
ORDER BY avg_stop_minutes DESC;
"""

# 3️⃣ Run the query
results = pd.read_sql(query, engine)

# 4️⃣ Inspect
print(results)

  violation  avg_stop_minutes
0     Other             20.45
1  Seatbelt             20.16
2  Speeding             20.15
3       DUI             20.09
4    Signal             20.06


In [ ]:
# 8. Are stops during the night more likely to lead to arrests?
query = """
SELECT
    CASE 
        WHEN EXTRACT(HOUR FROM stop_datetime) >= 20 OR EXTRACT(HOUR FROM stop_datetime) < 6 
        THEN 'Night'
        ELSE 'Day'
    END AS time_of_day,
    ROUND(AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100, 2) AS arrest_rate_percentage,
    COUNT(*) AS total_stops
FROM traffic_logs
GROUP BY time_of_day
ORDER BY arrest_rate_percentage DESC;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  time_of_day  arrest_rate_percentage  total_stops
0         Day                   50.49        38178
1       Night                   49.60        27360


In [47]:
# 9.Which violations are most associated with searches or arrests?
query = """
SELECT
    violation,
    ROUND(AVG(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) * 100, 2) AS search_rate,
    ROUND(AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100, 2) AS arrest_rate,
    COUNT(*) AS total_stops
FROM traffic_logs
GROUP BY violation
ORDER BY (
    AVG(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) * 100 +
    AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100
) DESC
LIMIT 10;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  violation  search_rate  arrest_rate  total_stops
0  Seatbelt        50.45        50.53        13007
1  Speeding        49.98        50.17        13150
2       DUI        49.57        50.55        13075
3    Signal        49.64        49.95        13112
4     Other        49.84        49.39        13194


In [ ]:
# 10.Which violations are most common among younger drivers (<25)?
query = """
SELECT
    violation,
    COUNT(*) AS total_count
FROM traffic_logs
WHERE driver_age < 25
GROUP BY violation
ORDER BY total_count DESC
LIMIT 10;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  violation  total_count
0  Speeding         1476
1    Signal         1427
2     Other         1422
3  Seatbelt         1420
4       DUI         1359


In [ ]:
# 11.Is there a violation that rarely results in search or arrest?
query = """
SELECT
    violation,
    ROUND(AVG(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) * 100, 2) AS search_rate,
    ROUND(AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100, 2) AS arrest_rate,
    COUNT(*) AS total_count
FROM traffic_logs
GROUP BY violation
ORDER BY (
    AVG(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) * 100 +
    AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100
) ASC
LIMIT 10;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  violation  search_rate  arrest_rate  total_count
0     Other        49.84        49.39        13194
1    Signal        49.64        49.95        13112
2       DUI        49.57        50.55        13075
3  Speeding        49.98        50.17        13150
4  Seatbelt        50.45        50.53        13007


In [ ]:
# 12.Which countries report the highest rate of drug-related stops?
query = """
SELECT
    country_name,
    ROUND(AVG(CASE WHEN drugs_related_stop = TRUE THEN 1 ELSE 0 END) * 100, 2) AS drug_related_rate,
    COUNT(*) AS total_stops
FROM traffic_logs
GROUP BY country_name
ORDER BY drug_related_rate DESC
LIMIT 10;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  country_name  drug_related_rate  total_stops
0          USA              50.37        21632
1       Canada              50.10        21908
2        India              49.54        21998


In [52]:
# 13.What is the arrest rate by country and violation?
query = """
SELECT
    country_name,
    violation,
    ROUND(AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100, 2) AS arrest_rate,
    COUNT(*) AS total_stops
FROM traffic_logs
GROUP BY country_name, violation
ORDER BY arrest_rate DESC
LIMIT 10;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  country_name violation  arrest_rate  total_stops
0       Canada       DUI        51.13         4414
1        India       DUI        50.84         4420
2        India  Speeding        50.76         4358
3        India    Signal        50.65         4444
4          USA  Seatbelt        50.58         4241
5        India  Seatbelt        50.56         4345
6       Canada  Seatbelt        50.46         4421
7       Canada     Other        50.29         4371
8          USA    Signal        49.99         4341
9       Canada  Speeding        49.94         4375


In [ ]:
# 14.Which country has the most stops with search conducted?
query = """
SELECT
    country_name,
    COUNT(*) AS total_searches
FROM traffic_logs
WHERE search_conducted = TRUE
GROUP BY country_name
ORDER BY total_searches DESC
LIMIT 5;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  country_name  total_searches
0       Canada           11020
1        India           10959
2          USA           10722


### complex

In [ ]:
# 1.Yearly Breakdown of Stops and Arrests by Country (Using Subquery and Window Functions)
query = """
WITH yearly_data AS (
    SELECT
        country_name,
        EXTRACT(YEAR FROM stop_datetime) AS year,
        COUNT(*) AS total_stops,
        SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS total_arrests
    FROM traffic_logs
    GROUP BY country_name, EXTRACT(YEAR FROM stop_datetime)
)
SELECT
    country_name,
    year,
    total_stops,
    total_arrests,
    ROUND((total_arrests::DECIMAL / total_stops) * 100, 2) AS arrest_rate_percentage,
    RANK() OVER (PARTITION BY year ORDER BY total_stops DESC) AS rank_by_year
FROM yearly_data
ORDER BY year, rank_by_year;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  country_name    year  total_stops  total_arrests  arrest_rate_percentage  \
0        India  2020.0        21998          11091                   50.42   
1       Canada  2020.0        21908          10999                   50.21   
2          USA  2020.0        21632          10756                   49.72   

   rank_by_year  
0             1  
1             2  
2             3  


In [55]:
# 2.Driver Violation Trends Based on Age and Race (Join with Subquery)
query = """
WITH age_groups AS (
    SELECT
        CASE
            WHEN driver_age < 25 THEN 'Under 25'
            WHEN driver_age BETWEEN 25 AND 40 THEN '25–40'
            ELSE 'Over 40'
        END AS age_group,
        country_name,
        violation,
        COUNT(*) AS total_count
    FROM traffic_logs
    GROUP BY 1, 2, 3
)

SELECT
    age_group,
    country_name,
    violation,
    total_count,
    RANK() OVER (PARTITION BY age_group, country_name ORDER BY total_count DESC) AS rank_in_group
FROM age_groups
ORDER BY age_group, country_name, rank_in_group
LIMIT 20;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

   age_group country_name violation  total_count  rank_in_group
0      25–40       Canada  Seatbelt         1170              1
1      25–40       Canada     Other         1128              2
2      25–40       Canada       DUI         1108              3
3      25–40       Canada  Speeding         1087              4
4      25–40       Canada    Signal         1062              5
5      25–40        India    Signal         1164              1
6      25–40        India  Speeding         1134              2
7      25–40        India       DUI         1133              3
8      25–40        India     Other         1117              4
9      25–40        India  Seatbelt         1072              5
10     25–40          USA  Speeding         1126              1
11     25–40          USA    Signal         1089              2
12     25–40          USA  Seatbelt         1089              2
13     25–40          USA       DUI         1085              4
14     25–40          USA     Other     

In [56]:
# 3.Time Period Analysis of Stops (Joining with Date Functions) , Number of Stops by Year,Month, Hour of the Day
query = """
SELECT
    EXTRACT(YEAR FROM stop_datetime) AS year,
    EXTRACT(MONTH FROM stop_datetime) AS month,
    EXTRACT(HOUR FROM stop_datetime) AS hour,
    COUNT(*) AS total_stops
FROM traffic_logs
GROUP BY 1, 2, 3
ORDER BY year, month, hour;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

      year  month  hour  total_stops
0   2020.0    1.0   0.0         1860
1   2020.0    1.0   1.0         1860
2   2020.0    1.0   2.0         1860
3   2020.0    1.0   3.0         1860
4   2020.0    1.0   4.0         1860
5   2020.0    1.0   5.0         1860
6   2020.0    1.0   6.0         1860
7   2020.0    1.0   7.0         1860
8   2020.0    1.0   8.0         1860
9   2020.0    1.0   9.0         1860
10  2020.0    1.0  10.0         1860
11  2020.0    1.0  11.0         1860
12  2020.0    1.0  12.0         1860
13  2020.0    1.0  13.0         1860
14  2020.0    1.0  14.0         1860
15  2020.0    1.0  15.0         1860
16  2020.0    1.0  16.0         1860
17  2020.0    1.0  17.0         1860
18  2020.0    1.0  18.0         1860
19  2020.0    1.0  19.0         1860
20  2020.0    1.0  20.0         1860
21  2020.0    1.0  21.0         1860
22  2020.0    1.0  22.0         1860
23  2020.0    1.0  23.0         1860
24  2020.0    2.0   0.0          900
25  2020.0    2.0   1.0          900
2

In [57]:
# 4.Violations with High Search and Arrest Rates (Window Function)
query = """
WITH stats AS (
    SELECT
        violation,
        ROUND(AVG(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) * 100, 2) AS search_rate,
        ROUND(AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100, 2) AS arrest_rate
    FROM traffic_logs
    GROUP BY violation
),
ranked AS (
    SELECT
        violation,
        search_rate,
        arrest_rate,
        (search_rate + arrest_rate) AS combined_rate,
        RANK() OVER (ORDER BY (search_rate + arrest_rate) DESC) AS rate_rank
    FROM stats
)
SELECT *
FROM ranked
ORDER BY rate_rank
LIMIT 10;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  violation  search_rate  arrest_rate  combined_rate  rate_rank
0  Seatbelt        50.45        50.53         100.98          1
1  Speeding        49.98        50.17         100.15          2
2       DUI        49.57        50.55         100.12          3
3    Signal        49.64        49.95          99.59          4
4     Other        49.84        49.39          99.23          5


In [58]:
# 5.Driver Demographics by Country (Age, Gender, and Race)
query = """
WITH age_groups AS (
    SELECT
        country_name,
        CASE
            WHEN driver_age < 25 THEN 'Under 25'
            WHEN driver_age BETWEEN 25 AND 40 THEN '25-40'
            ELSE 'Over 40'
        END AS age_group,
        driver_gender,
        COUNT(*) AS total_count
    FROM traffic_logs
    GROUP BY country_name, age_group, driver_gender
)
SELECT
    country_name,
    age_group,
    driver_gender,
    total_count,
    ROUND(100.0 * total_count / SUM(total_count) OVER (PARTITION BY country_name), 2) AS percentage_in_country
FROM age_groups
ORDER BY country_name, age_group, driver_gender;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

   country_name age_group driver_gender  total_count  percentage_in_country
0        Canada     25-40        Female         2749                  12.55
1        Canada     25-40          Male         2806                  12.81
2        Canada   Over 40        Female         7037                  32.12
3        Canada   Over 40          Male         6922                  31.60
4        Canada  Under 25        Female         1211                   5.53
5        Canada  Under 25          Male         1183                   5.40
6         India     25-40        Female         2825                  12.84
7         India     25-40          Male         2795                  12.71
8         India   Over 40        Female         7054                  32.07
9         India   Over 40          Male         6972                  31.69
10        India  Under 25        Female         1164                   5.29
11        India  Under 25          Male         1188                   5.40
12          

In [59]:
# 6.Top 5 Violations with Highest Arrest Rates
query = """
SELECT
    violation,
    ROUND(AVG(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) * 100, 2) AS arrest_rate,
    COUNT(*) AS total_stops
FROM traffic_logs
GROUP BY violation
ORDER BY arrest_rate DESC
LIMIT 5;
"""

# Run the query
results = pd.read_sql(query, engine)

print(results)

  violation  arrest_rate  total_stops
0       DUI        50.55        13075
1  Seatbelt        50.53        13007
2  Speeding        50.17        13150
3    Signal        49.95        13112
4     Other        49.39        13194
